In [1]:
! pip install neo4j

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 5.0 MB/s eta 0:00:00a 0:00:01
  Using cached http://mirrors.aliyun.com/pypi/packages/81/c4/34e93fe5f5429d7570ec1fa436f1986fb1f00c3e0f43a589fe2bbcd22c3f/pytz-2025.2-py2.py3-none-any.whl (509 kB)


In [2]:
pip install pandas

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 12.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 20.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
! pip install tqdm

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [5]:
from neo4j import GraphDatabase
import pandas as pd
from tqdm import tqdm  # 进度条工具（可选）

# Neo4j AuraDB 连接配置
URI = "neo4j+s://68689680.databases.neo4j.io"
USER = "neo4j"
PASSWORD = "IubLSzByNDoUMEg_LCBZe9zQ_DXZuzNubNX8QL_lyO0"

def create_constraint(session):
    """创建OrderID的唯一性约束"""
    session.run("""
    CREATE CONSTRAINT order_id_unique IF NOT EXISTS 
    FOR (o:Order) REQUIRE o.orderID IS UNIQUE
    """)
    print("已确保OrderID的唯一性约束")

def import_orders():
    driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))
    
    try:
        # 1. 首先创建约束
        with driver.session() as session:
            create_constraint(session)
        
        # 2. 读取并预处理数据
        df = pd.read_csv("exported_data/orders.csv")
        df = df.where(pd.notnull(df), None)  # 处理空值
        
        # 3. 批量导入（带冲突处理）
        with driver.session() as session:
            # 使用UNWIND + MERGE确保不重复插入
            query = """
            UNWIND $orders AS order
            MERGE (o:Order {orderID: order.OrderID})
            ON CREATE SET
                o.orderDate = order.OrderDate,
                o.requiredDate = order.RequiredDate,
                o.shippedDate = order.ShippedDate,
                o.freight = toFloat(order.Freight),
                o.shipName = order.ShipName,
                o.shipAddress = order.ShipAddress,
                o.shipCity = order.ShipCity,
                o.shipRegion = order.ShipRegion,
                o.shipPostalCode = order.ShipPostalCode,
                o.shipCountry = order.ShipCountry
            """
            
            # 分批导入（每批1000条）
            batch_size = 1000
            for i in tqdm(range(0, len(df), batch_size), desc="导入进度"):
                batch = df.iloc[i:i+batch_size].to_dict('records')
                session.run(query, parameters={"orders": batch})
            
            print(f"成功导入 {len(df)} 条订单数据（已自动去重）")

    except Exception as e:
        print(f"错误: {str(e)}")
    finally:
        driver.close()

if __name__ == "__main__":
    import_orders()

已确保OrderID的唯一性约束


导入进度: 100%|██████████| 1/1 [00:00<00:00,  1.10it/s]

成功导入 100 条订单数据（已自动去重）


In [6]:
from neo4j import GraphDatabase
import pandas as pd

# Neo4j AuraDB 连接配置（注意：建议将密码移出代码，使用环境变量）
URI = "neo4j+s://68689680.databases.neo4j.io"
USER = "neo4j"
PASSWORD = "IubLSzByNDoUMEg_LCBZe9zQ_DXZuzNubNX8QL_lyO0"  # 请尽快修改密码并避免公开

def create_shipper_constraint(session):
    """创建ShipperID的唯一性约束"""
    session.run("""
    CREATE CONSTRAINT shipper_id_unique IF NOT EXISTS 
    FOR (s:Shipper) REQUIRE s.ShipperID IS UNIQUE
    """)
    print("已确保ShipperID的唯一性约束")

def import_shippers():
    # 初始化驱动
    driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))
    
    try:
        # 1. 读取CSV文件
        df = pd.read_csv("exported_data/shippers.csv")
        
        # 2. 数据预处理：提取所需列并去重
        shipper_df = df[['ShipperID', 'CompanyName', 'Phone']].drop_duplicates()
        
        # 3. 创建约束并导入数据
        with driver.session() as session:
            # 首先创建唯一约束
            create_shipper_constraint(session)
            
            # 使用MERGE避免重复插入
            query = """
            UNWIND $shippers AS shipper
            MERGE (s:Shipper {ShipperID: shipper.ShipperID})
            ON CREATE SET
                s.CompanyName = shipper.CompanyName,
                s.Phone = shipper.Phone
            """
            
            # 转换为字典列表并批量提交
            shippers_data = shipper_df.to_dict('records')
            result = session.run(query, parameters={"shippers": shippers_data})
            
            # 获取统计信息
            summary = result.consume()
            print(f"成功处理 {len(shippers_data)} 条货运公司数据")
            print(f"实际新增节点: {summary.counters.nodes_created}")
            print(f"跳过重复节点: {len(shippers_data) - summary.counters.nodes_created}")

    except Exception as e:
        print(f"导入过程中发生错误: {e}")
    finally:
        driver.close()
        print("数据库连接已安全关闭")

if __name__ == "__main__":
    import_shippers()

已确保ShipperID的唯一性约束
成功处理 10 条货运公司数据
实际新增节点: 10
跳过重复节点: 0
数据库连接已安全关闭


In [7]:
from neo4j import GraphDatabase
import pandas as pd
from datetime import datetime
import os

# 安全提示：建议使用环境变量存储敏感信息
# 在运行前请设置环境变量：export NEO4J_PASSWORD="your_password"
URI = "neo4j+s://68689680.databases.neo4j.io"
USER = "neo4j"
PASSWORD = os.getenv("NEO4J_PASSWORD", "IubLSzByNDoUMEg_LCBZe9zQ_DXZuzNubNX8QL_lyO0")

def create_employee_constraint(session):
    """创建EmployeeID的唯一性约束"""
    session.run("""
    CREATE CONSTRAINT employee_id_unique IF NOT EXISTS
    FOR (e:Employee) REQUIRE e.EmployeeID IS UNIQUE
    """)
    print("✅ 已确保EmployeeID的唯一性约束")

def clean_data(df):
    """数据清洗函数"""
    # 处理日期格式（假设CSV中是字符串格式如'1994-05-23'）
    date_columns = ['BirthDate', 'HireDate']
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce').dt.strftime('%Y-%m-%d')
    
    # 处理空值
    df = df.replace({pd.NA: None, '': None})
    return df

def import_employees():
    driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))
    
    try:
        print("🔄 正在读取employee.csv文件...")
        df = pd.read_csv("exported_data/employees.csv")
        
        # 数据清洗
        df = clean_data(df)
        
        with driver.session() as session:
            # 1. 创建约束
            create_employee_constraint(session)
            
            # 2. 准备批量导入查询
            query = """
            UNWIND $employees AS emp
            MERGE (e:Employee {EmployeeID: emp.EmployeeID})
            ON CREATE SET
                e.LastName = emp.LastName,
                e.FirstName = emp.FirstName,
                e.Title = emp.Title,
                e.BirthDate = date(emp.BirthDate),
                e.HireDate = date(emp.HireDate),
                e.Address = emp.Address,
                e.City = emp.City,
                e.Country = emp.Country,
                e.HomePhone = emp.HomePhone,
                e.Notes = emp.Notes
            """
            
            # 3. 分批导入（每批500条）
            batch_size = 500
            total = len(df)
            print(f"📊 共发现 {total} 条员工记录")
            
            for i in range(0, total, batch_size):
                batch = df.iloc[i:i+batch_size].to_dict('records')
                session.run(query, parameters={"employees": batch})
                print(f"⏳ 已处理 {min(i+batch_size, total)}/{total} 条")
            
            print("🎉 员工数据导入完成！")

    except Exception as e:
        print(f"❌ 导入过程中发生错误: {str(e)}")
    finally:
        driver.close()
        print("🔒 数据库连接已安全关闭")

if __name__ == "__main__":
    # 安全检查
    if not PASSWORD:
        print("⚠️ 错误：未检测到数据库密码！请设置NEO4J_PASSWORD环境变量")
        exit(1)
    
    import_employees()

🔄 正在读取employee.csv文件...
✅ 已确保EmployeeID的唯一性约束
📊 共发现 3 条员工记录
⏳ 已处理 3/3 条
🎉 员工数据导入完成！
🔒 数据库连接已安全关闭


In [9]:
from neo4j import GraphDatabase
import pandas as pd
import os
from datetime import datetime

# 安全配置（建议通过环境变量设置密码）
NEO4J_URI = os.getenv("NEO4J_URI", "neo4j+s://68689680.databases.neo4j.io")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASSWORD ="IubLSzByNDoUMEg_LCBZe9zQ_DXZuzNubNX8QL_lyO0"  # 必须通过环境变量设置

def validate_env():
    """验证环境变量配置"""
    if not NEO4J_PASSWORD:
        raise ValueError("未检测到数据库密码！请设置NEO4J_PASSWORD环境变量")

class Neo4jCustomerImporter:
    def __init__(self):
        self.driver = GraphDatabase.driver(
            NEO4J_URI,
            auth=(NEO4J_USER, NEO4J_PASSWORD),
            max_connection_lifetime=30*60,
            connection_timeout=15  # 15秒连接超时
        )
    
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.driver.close()
    
    def create_constraints(self):
        """创建数据库约束"""
        with self.driver.session() as session:
            session.execute_write(lambda tx: tx.run("""
                CREATE CONSTRAINT customer_id_unique IF NOT EXISTS
                FOR (c:Customer) REQUIRE c.CustomerID IS UNIQUE
                """))
            print("数据库约束已就绪")

    def preprocess_data(self, file_path):
        """数据预处理"""
        df = pd.read_csv(file_path)
        
        # 列名标准化处理
        df.columns = df.columns.str.strip()
        
        # 空值处理
        df = df.replace({pd.NA: None, '': None, 'NULL': None})
        
        # 字符串字段清理
        string_cols = ['CompanyName', 'ContactName', 'ContactTitle']
        for col in string_cols:
            if col in df.columns:
                df[col] = df[col].str.strip()
        
        # 验证必需字段
        required_fields = ['CustomerID', 'CompanyName']
        missing_fields = [f for f in required_fields if f not in df.columns]
        if missing_fields:
            raise ValueError(f"CSV文件缺少必需字段: {missing_fields}")
        
        return df

    def import_data(self, df):
        """执行数据导入"""
        total = len(df)
        print(f"开始导入 {total} 条客户记录...")
        
        with self.driver.session() as session:
            # 使用自动提交事务（auto-commit）提高性能
            result = session.run("""
                UNWIND $customers AS row
                MERGE (c:Customer {CustomerID: row.CustomerID})
                ON CREATE SET
                    c.CompanyName = row.CompanyName,
                    c.ContactName = row.ContactName,
                    c.ContactTitle = row.ContactTitle,
                    c.Address = row.Address,
                    c.City = row.City,
                    c.Region = row.Region,
                    c.PostalCode = row.PostalCode,
                    c.Country = row.Country,
                    c.Phone = row.Phone,
                    c.Fax = row.Fax
                RETURN count(c)
                """, {"customers": df.to_dict('records')})
            
            inserted = result.single()[0]
            print(f"导入完成！新增 {inserted} 条记录")
            if inserted < total:
                print(f"注意：跳过 {total - inserted} 条重复记录")

def main():
    try:
        validate_env()
        
        with Neo4jCustomerImporter() as importer:
            # 1. 创建约束
            importer.create_constraints()
            
            # 2. 加载并预处理数据
            df = importer.preprocess_data("exported_data/customers.csv")
            
            # 3. 执行导入
            importer.import_data(df)
            
    except Exception as e:
        print(f"错误: {str(e)}")
        if isinstance(e, pd.errors.EmptyDataError):
            print("请检查CSV文件是否为空或格式不正确")
        elif isinstance(e, FileNotFoundError):
            print("文件路径错误，请确认exported_data/customers.csv是否存在")
        return 1
    return 0

if __name__ == "__main__":
    exit(main())

数据库约束已就绪
开始导入 20 条客户记录...
导入完成！新增 20 条记录


In [1]:
from neo4j import GraphDatabase
import pandas as pd
import os
from tqdm import tqdm  # 进度条支持

# 安全配置（强烈建议通过环境变量设置密码）
NEO4J_URI = "neo4j+s://68689680.databases.neo4j.io"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD") or "IubLSzByNDoUMEg_LCBZe9zQ_DXZuzNubNX8QL_lyO0"

class SupplierImporter:
    def __init__(self):
        # 配置连接池（优化性能）
        self.driver = GraphDatabase.driver(
            NEO4J_URI,
            auth=(NEO4J_USER, NEO4J_PASSWORD),
            max_connection_pool_size=10,
            connection_timeout=30  # 30秒连接超时
        )
    
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.driver.close()
    
    def setup_constraints(self):
        """设置数据库约束（幂等操作）"""
        with self.driver.session(database="neo4j") as session:
            result = session.run("""
                CREATE CONSTRAINT supplier_id_unique IF NOT EXISTS
                FOR (s:Supplier) REQUIRE s.SupplierID IS UNIQUE
                """)
            summary = result.consume()
            print("✅ 供应商ID唯一约束已生效")

    def preprocess_data(self, file_path):
        """数据预处理管道"""
        try:
            df = pd.read_csv(file_path, keep_default_na=False)
            
            # 字段标准化
            df.columns = df.columns.str.strip().str.replace(' ', '')
            
            # 空值处理（将空字符串和NaN统一为None）
            df = df.replace(['', 'NULL', 'null', pd.NA], None)
            
            # URL字段预处理
            if 'HomePage' in df.columns:
                df['HomePage'] = df['HomePage'].apply(
                    lambda x: x if pd.isna(x) or x.startswith(('http://', 'https://')) 
                    else f'http://{x}'
                )
            
            # 验证必需字段
            required_fields = {'SupplierID', 'CompanyName'}
            missing_fields = required_fields - set(df.columns)
            if missing_fields:
                raise ValueError(f"缺少必需字段: {missing_fields}")
            
            return df
        
        except Exception as e:
            print(f"❌ 数据预处理失败: {str(e)}")
            raise

    def batch_import(self, df, batch_size=500):
        """高效批量导入"""
        total_records = len(df)
        print(f"📊 共发现 {total_records} 条供应商记录")
        
        with self.driver.session(database="neo4j") as session:
            # 使用UNWIND批量操作
            query = """
            UNWIND $batch AS supplier
            MERGE (s:Supplier {SupplierID: supplier.SupplierID})
            ON CREATE SET
                s.CompanyName = supplier.CompanyName,
                s.ContactName = supplier.ContactName,
                s.ContactTitle = supplier.ContactTitle,
                s.Address = supplier.Address,
                s.City = supplier.City,
                s.Region = supplier.Region,
                s.PostalCode = supplier.PostalCode,
                s.Country = supplier.Country,
                s.Phone = supplier.Phone,
                s.Fax = supplier.Fax,
                s.HomePage = supplier.HomePage
            RETURN count(s)
            """
            
            inserted = 0
            with tqdm(total=total_records, desc="导入进度") as pbar:
                for i in range(0, total_records, batch_size):
                    batch = df.iloc[i:i+batch_size].to_dict('records')
                    result = session.run(query, {"batch": batch})
                    inserted += result.single()[0]
                    pbar.update(len(batch))
            
            print(f"🎉 导入完成！新增 {inserted} 条记录")
            if duplicates := total_records - inserted:
                print(f"⚠️  跳过 {duplicates} 条重复记录")

def main():
    try:
        # 初始化导入器（自动管理连接）
        with SupplierImporter() as importer:
            # 1. 设置约束
            importer.setup_constraints()
            
            # 2. 加载并预处理数据
            df = importer.preprocess_data("exported_data/suppliers.csv")
            
            # 3. 执行批量导入
            importer.batch_import(df, batch_size=1000)
            
    except Exception as e:
        print(f"❌ 导入失败: {str(e)}")
        return 1
    return 0

if __name__ == "__main__":
    # 安全警告检查
    if NEO4J_PASSWORD == "IubLSzByNDoUMEg_LCBZe9zQ_DXZuzNubNX8QL_lyO0":
        print("⚠️ 警告：正在使用示例密码，请立即更改为自己的密码！")
    
    exit(main())

⚠️ 警告：正在使用示例密码，请立即更改为自己的密码！
✅ 供应商ID唯一约束已生效
📊 共发现 15 条供应商记录


导入进度: 100%|██████████| 15/15 [00:00<00:00, 52.01it/s]

🎉 导入完成！新增 15 条记录


In [1]:
from neo4j import GraphDatabase
import pandas as pd
import os
from tqdm import tqdm  # 进度条支持

# 安全配置（建议通过环境变量设置密码）
NEO4J_URI = "neo4j+s://68689680.databases.neo4j.io"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD") or "IubLSzByNDoUMEg_LCBZe9zQ_DXZuzNubNX8QL_lyO0"

class CategoryImporter:
    def __init__(self):
        self.driver = GraphDatabase.driver(
            NEO4J_URI,
            auth=(NEO4J_USER, NEO4J_PASSWORD),
            max_connection_pool_size=5,
            connection_timeout=15
        )
    
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.driver.close()
    
    def setup_constraints(self):
        """创建CategoryID唯一约束"""
        with self.driver.session() as session:
            result = session.run("""
                CREATE CONSTRAINT category_id_unique IF NOT EXISTS
                FOR (c:Category) REQUIRE c.CategoryID IS UNIQUE
                """)
            print("✅ 分类ID唯一约束已创建")

    def preprocess_data(self, file_path):
        """数据预处理"""
        try:
            df = pd.read_csv(file_path)
            
            # 列名标准化
            df.columns = df.columns.str.strip()
            
            # 空值处理
            df = df.replace(['', 'NULL', pd.NA], None)
            
            # 验证必需字段
            required = {'CategoryID', 'CategoryName'}
            missing = required - set(df.columns)
            if missing:
                raise ValueError(f"缺少必需字段: {missing}")
            
            return df
        
        except Exception as e:
            print(f"❌ 数据预处理失败: {str(e)}")
            raise

    def import_categories(self, df):
        """执行数据导入"""
        total = len(df)
        print(f"📋 共发现 {total} 条分类记录")
        
        with self.driver.session() as session:
            # 高效批量导入查询
            query = """
            UNWIND $categories AS cat
            MERGE (c:Category {CategoryID: cat.CategoryID})
            ON CREATE SET
                c.CategoryName = cat.CategoryName,
                c.Description = cat.Description
            RETURN count(c)
            """
            
            # 分批处理（每批1000条）
            inserted = 0
            batch_size = 1000
            
            with tqdm(total=total, desc="导入进度") as pbar:
                for i in range(0, total, batch_size):
                    batch = df.iloc[i:i+batch_size].to_dict('records')
                    result = session.run(query, {"categories": batch})
                    inserted += result.single()[0]
                    pbar.update(len(batch))
            
            print(f"🎉 成功导入 {inserted} 条分类数据")
            if duplicates := total - inserted:
                print(f"⚠️  跳过 {duplicates} 条重复记录")

def main():
    try:
        # 密码安全检查
        if "IubLSzByNDoUMEg" in NEO4J_PASSWORD:
            print("⚠️ 警告：请立即更改默认密码！")
        
        with CategoryImporter() as importer:
            # 1. 设置约束
            importer.setup_constraints()
            
            # 2. 加载数据
            df = importer.preprocess_data("exported_data/categories.csv")
            
            # 3. 执行导入
            importer.import_categories(df)
            
    except Exception as e:
        print(f"❌ 导入失败: {str(e)}")
        return 1
    return 0

if __name__ == "__main__":
    exit(main())

⚠️ 警告：请立即更改默认密码！
✅ 分类ID唯一约束已创建
📋 共发现 20 条分类记录


导入进度: 100%|██████████| 20/20 [00:00<00:00, 75.85it/s]

🎉 成功导入 20 条分类数据


In [1]:
from neo4j import GraphDatabase
import pandas as pd
import os
from tqdm import tqdm

# 安全配置（生产环境建议使用环境变量）
NEO4J_URI = "neo4j+s://68689680.databases.neo4j.io"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD") or "IubLSzByNDoUMEg_LCBZe9zQ_DXZuzNubNX8QL_lyO0"

class ProductImporter:
    def __init__(self):
        self.driver = GraphDatabase.driver(
            NEO4J_URI,
            auth=(NEO4J_USER, NEO4J_PASSWORD),
            max_connection_pool_size=8,
            connection_acquisition_timeout=30
        )
    
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.driver.close()
    
    def setup_schema(self):
        """创建唯一约束和索引"""
        with self.driver.session() as session:
            # 创建唯一约束
            session.run("""
                CREATE CONSTRAINT product_id_unique IF NOT EXISTS
                FOR (p:Product) REQUIRE p.ProductID IS UNIQUE
                """)
            
            # 为常用查询字段创建索引
            session.run("""
                CREATE INDEX product_name_index IF NOT EXISTS
                FOR (p:Product) ON (p.ProductName)
                """)
            
            print("✅ 数据库约束和索引已就绪")

    def preprocess_products(self, file_path):
        """数据预处理管道"""
        try:
            df = pd.read_csv(file_path, keep_default_na=False)
            
            # 列名标准化
            df.columns = df.columns.str.strip()
            
            # 类型转换
            numeric_cols = ['UnitPrice', 'UnitsInStock', 'UnitsOnOrder', 'ReorderLevel']
            for col in numeric_cols:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # 布尔值处理
            if 'Discontinued' in df.columns:
                df['Discontinued'] = df['Discontinued'].astype(bool)
            
            # 空值处理
            df = df.replace(['', 'NULL', 'null', pd.NA], None)
            
            # 验证必需字段
            required = {'ProductID', 'ProductName'}
            missing = required - set(df.columns)
            if missing:
                raise ValueError(f"CSV文件缺少必需字段: {missing}")
            
            return df
        
        except Exception as e:
            print(f"❌ 数据预处理错误: {str(e)}")
            raise

    def batch_import(self, df, batch_size=2000):
        """高效批量导入"""
        total = len(df)
        print(f"📊 共发现 {total} 条产品记录")
        
        with self.driver.session() as session:
            # 使用参数化查询防止注入
            query = """
            UNWIND $products AS p
            MERGE (prod:Product {ProductID: p.ProductID})
            ON CREATE SET
                prod.ProductName = p.ProductName,
                prod.QuantityPerUnit = p.QuantityPerUnit,
                prod.UnitPrice = toFloat(p.UnitPrice),
                prod.UnitsInStock = toInteger(p.UnitsInStock),
                prod.UnitsOnOrder = toInteger(p.UnitsOnOrder),
                prod.ReorderLevel = toInteger(p.ReorderLevel),
                prod.Discontinued = p.Discontinued
            RETURN count(prod)
            """
            
            inserted = 0
            with tqdm(total=total, desc="导入进度", unit="product") as pbar:
                for i in range(0, total, batch_size):
                    batch = df.iloc[i:i+batch_size].to_dict('records')
                    result = session.run(query, {"products": batch})
                    inserted += result.single()[0]
                    pbar.update(len(batch))
            
            print(f"✅ 成功导入 {inserted} 条产品数据")
            if duplicates := total - inserted:
                print(f"⚠️  跳过 {duplicates} 条重复记录")

def main():
    try:
        # 密码安全检查
        if NEO4J_PASSWORD.startswith("IubLSz"):
            print("⚠️ 安全警告：请立即更改默认数据库密码！")
        
        with ProductImporter() as importer:
            # 1. 初始化数据库schema
            importer.setup_schema()
            
            # 2. 加载并预处理数据
            df = importer.preprocess_products("exported_data/products.csv")
            
            # 3. 执行批量导入
            importer.batch_import(df, batch_size=1500)
            
    except Exception as e:
        print(f"❌ 致命错误: {str(e)}")
        return 1
    return 0

if __name__ == "__main__":
    exit(main())

⚠️ 安全警告：请立即更改默认数据库密码！
✅ 数据库约束和索引已就绪
📊 共发现 10 条产品记录


导入进度: 100%|██████████| 10/10 [00:00<00:00, 33.88product/s]

✅ 成功导入 10 条产品数据


Imported 15 nodes from data/neo4j_admin/supplier_nodes.csv with label Supplier
Imported 10 nodes from data/neo4j_admin/shipper_nodes.csv with label Shipper
Imported 30 nodes from data/neo4j_admin/review_nodes.csv with label Review
Imported 10 nodes from data/neo4j_admin/product_nodes.csv with label Product
Imported 100 nodes from data/neo4j_admin/order_nodes.csv with label Order
Imported 3 nodes from data/neo4j_admin/employee_nodes.csv with label Employee
Imported 20 nodes from data/neo4j_admin/customer_nodes.csv with label Customer
Imported 20 nodes from data/neo4j_admin/category_nodes.csv with label Category


/tmp/ipykernel_12341/3191131646.py:78: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  start_id = row[0]
/tmp/ipykernel_12341/3191131646.py:79: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  end_id = row[1]


ValueError: The truth value of a Index is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().